# employee-evac-map: 従業員の自宅と避難情報をマップで可視化

従業員の自宅と避難情報が発表されている地域の関係を、Python/Jupyter Notebook を使って可視化するための分析ツール。従業員の住所とは関係なく、避難情報が発表された地区の地理的・時系列的な広がりを可視化するためにも使用可能。

詳しい使用方法は、[README.md](README.md)参照。

> ここでサンプルデータとして使用している従業員リストの名前や番地以下の住所は[Faker](https://faker.readthedocs.io/en/master/)で作成した架空のものです。実在の人物・住所とは関係ありません。


## 変数の設定

ここで定義する変数の値は適宜変更すること。


In [115]:
import os

# データの保存先ディレクトリ
DATA_DIR = "data"

# 地図の保存先ディレクトリ
MAP_DIR = "maps"

# 国土数値情報ダウンロードサイトから取得した位置参照情報のCSVファイルのパス
BLOCK_POSITIONS_FILE_PATH = os.path.join(DATA_DIR, "akita-2022-ichijoho.csv")

# 従業員の住所一覧のCSVファイルのパス
EMPLOYEE_ADDRESS_FILE_PATH = os.path.join(DATA_DIR, "mock_employee_list.csv")

# 避難情報のCSVファイルのパス
EVAC_INFO_FILE_PATH = os.path.join(DATA_DIR, "akita_evac_all.csv")

# マッピングする時点一覧 yyyy-MM-dd hh:mm:ss
MAPPING_DATETIME_LIST = [
    "2023-07-15 09:00:00",
    "2023-07-15 11:00:00",
    "2023-07-15 13:00:00",
    "2023-07-15 15:00:00",
    "2023-07-15 17:00:00",
    "2023-07-15 19:00:00",
    "2023-07-16 05:00:00",
]

# 描画する地図の、中心地点の緯度・経度
MAP_CENTER = [39.716719, 140.129775]  # ここでは秋田駅

# 描画する地図の、初期のズームレベル
MAP_ZOOM_START = 11

# 描画する地図の様式
# https://python-visualization.github.io/folium/modules.html
MAP_TILES = "Stamen Terrain"  # "OpenStreetMap" "Stamen Terrain" "Stamen Toner" "Stamen Watercolor" "CartoDB positron" "CartoDB dark_matter"

# 描画する地図に表示する、会社の所在地などのマーカー
# 本社だけでなく、支社や営業所などを複数表示することもできる
# 逆に何も追加で表示したくない場合は、空のリストにする
MAP_MARKERS = [
    {
        "location": [39.719007, 140.102866],  # 緯度・経度。ここでは秋田県庁
        "popup": "秋田県庁",  # 地図上で表示される文字列
        "icon": "building",  # https://fontawesome.com/icons
        "color": "darkgreen",  # https://python-visualization.github.io/folium/modules.html
    },
    {
        "location": [39.800408, 140.218396],
        "popup": "旭川ダム",
        "icon": "water",
        "color": "darkgreen",
    },
]

# 描画する地図に表示する凡例
# [[datetime_string]]は、MAPPING_DATETIME_LISTの各要素の文字列に置換される
MAP_LEGEND = """<div style="position: fixed; bottom: 50px; left: 50px; width: 240px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white;
     ">&nbsp; 避難レベル ([[datetime_string]])<br>
     &nbsp; 3 (高齢者等避難) &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
     &nbsp; 4 (避難指示) &nbsp; <i class="fa fa-map-marker fa-2x" style="color:purple"></i><br>
     &nbsp; 5 (緊急安全確保) &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i>
      </div>
     """

## 事前に準備したファイルを読み込む

Pandas の data frame 形式で、事前に準備した CSV ファイルの情報を読み込む。


### 位置参照情報


In [116]:
import pandas as pd

# 国土数値情報ダウンロードサイトから取得した位置参照情報
blocks_df = pd.read_csv(BLOCK_POSITIONS_FILE_PATH, encoding="utf-8")

# 確認のため、最初の5行を表示
blocks_df.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,5,秋田県,5201,秋田市,52010001000,旭川清澄町,39.743039,140.128123,0,1
1,5,秋田県,5201,秋田市,52010002000,旭川新藤田西町,39.740677,140.126120,0,1
2,5,秋田県,5201,秋田市,52010003000,旭川新藤田東町,39.740510,140.129167,0,1
3,5,秋田県,5201,秋田市,52010004000,旭川南町,39.736556,140.128467,0,1
4,5,秋田県,5201,秋田市,52010005000,新屋朝日町,39.701936,140.087242,0,1


In [117]:
# 列名「大字町丁目名」を「BLOCK」に変更
blocks_df.rename(columns={"大字町丁目名": "BLOCK"}, inplace=True)

# 確認のため、再度最初の5行を表示
blocks_df.head()

,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,BLOCK,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,5,秋田県,5201,秋田市,52010001000,旭川清澄町,39.743039,140.128123,0,1
1,5,秋田県,5201,秋田市,52010002000,旭川新藤田西町,39.740677,140.126120,0,1
2,5,秋田県,5201,秋田市,52010003000,旭川新藤田東町,39.740510,140.129167,0,1
3,5,秋田県,5201,秋田市,52010004000,旭川南町,39.736556,140.128467,0,1
4,5,秋田県,5201,秋田市,52010005000,新屋朝日町,39.701936,140.087242,0,1


### 従業員の住所一覧


In [118]:
# 従業員の住所一覧
employee_address_df = pd.read_csv(EMPLOYEE_ADDRESS_FILE_PATH, encoding="utf-8")

# 確認のため、最初の5行を表示
employee_address_df.head()

,ID,NAME,ADDRESS,BLOCK
0,88666324,高橋 真綾,秋田市桜ガ丘二丁目50-67,桜ガ丘二丁目
1,85215597,村上 治,秋田市楢山共和町90-88,楢山共和町
2,69370697,田中 春香,秋田市茨島四丁目23-7,茨島四丁目
3,57453651,佐藤 七夏,秋田市楢山南新町上丁17-21,楢山南新町上丁
4,10586564,石川 舞,秋田市八橋南二丁目40-18,八橋南二丁目


In [119]:
# 従業員の住所一覧に、大字・町丁目単位で代表地点の位置参照情報を結合する
employee_address_with_block_df = pd.merge(
    employee_address_df, blocks_df, on=["BLOCK"], how="left"
)

# 確認のため、最初の5行を表示
employee_address_with_block_df.head()

,ID,NAME,ADDRESS,BLOCK,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,88666324,高橋 真綾,秋田市桜ガ丘二丁目50-67,桜ガ丘二丁目,5,秋田県,5201,秋田市,52010142002,39.704051,140.155681,0,3
1,85215597,村上 治,秋田市楢山共和町90-88,楢山共和町,5,秋田県,5201,秋田市,52010240000,39.703010,140.116457,0,1
2,69370697,田中 春香,秋田市茨島四丁目23-7,茨島四丁目,5,秋田県,5201,秋田市,52010265004,39.696752,140.101111,0,3
3,57453651,佐藤 七夏,秋田市楢山南新町上丁17-21,楢山南新町上丁,5,秋田県,5201,秋田市,52010248000,39.705353,140.121259,0,1
4,10586564,石川 舞,秋田市八橋南二丁目40-18,八橋南二丁目,5,秋田県,5201,秋田市,52010298002,39.720990,140.088677,0,3


In [120]:
# 位置参照情報が結合されていない行を確認する
employee_address_with_block_df[employee_address_with_block_df["緯度"].isnull()]

# 該当する行がなければok
# あれば、従業員一覧の大字町丁目名に誤字脱字や表記ゆれがないか確認する

,ID,NAME,ADDRESS,BLOCK,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,緯度,経度,原典資料コード,大字・字・丁目区分コード


### 発表された避難情報


In [121]:
# 発表された避難情報
evac_info_df = pd.read_csv(EVAC_INFO_FILE_PATH, encoding="utf-8")

# 確認のため、最初の5行を表示
evac_info_df.head()

,MAIL_TIMESTAMP,LEVEL,LEVEL_NAME,REASON,BLOCK_ORIGINAL,BLOCK
0,2023/7/15 7:25:00,3,高齢者等避難,洪水,牛島東六丁目,牛島東六丁目
1,2023/7/15 7:25:00,3,高齢者等避難,洪水,牛島東七丁目,牛島東七丁目
2,2023/7/15 7:25:00,3,高齢者等避難,洪水,牛島西二丁目,牛島西二丁目
3,2023/7/15 7:25:00,3,高齢者等避難,洪水,牛島西三丁目,牛島西三丁目
4,2023/7/15 7:25:00,3,高齢者等避難,洪水,牛島西四丁目,牛島西四丁目


In [122]:
# 避難情報のデータ型を確認
evac_info_df.dtypes

MAIL_TIMESTAMP    object
LEVEL              int64
LEVEL_NAME        object
REASON            object
BLOCK_ORIGINAL    object
BLOCK             object
dtype: object

In [123]:
# MAIL_TIMESTAMPを日付型に変換
evac_info_df["MAIL_TIMESTAMP"] = pd.to_datetime(evac_info_df["MAIL_TIMESTAMP"])

# 改めて、避難情報のデータ型を確認
evac_info_df.dtypes

MAIL_TIMESTAMP    datetime64[ns]
LEVEL                      int64
LEVEL_NAME                object
REASON                    object
BLOCK_ORIGINAL            object
BLOCK                     object
dtype: object

In [124]:
# BLOCK列をキーに、避難情報に位置参照情報を結合する
evac_info_with_block_df = pd.merge(evac_info_df, blocks_df, on=["BLOCK"], how="left")

# 確認のため、最初の5行を表示
evac_info_with_block_df.head()

,MAIL_TIMESTAMP,LEVEL,LEVEL_NAME,REASON,BLOCK_ORIGINAL,BLOCK,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,2023-07-15 07:25:00,3,高齢者等避難,洪水,牛島東六丁目,牛島東六丁目,5,秋田県,5201,秋田市,52010064006,39.694857,140.124936,0,3
1,2023-07-15 07:25:00,3,高齢者等避難,洪水,牛島東七丁目,牛島東七丁目,5,秋田県,5201,秋田市,52010064007,39.693858,140.133328,0,3
2,2023-07-15 07:25:00,3,高齢者等避難,洪水,牛島西二丁目,牛島西二丁目,5,秋田県,5201,秋田市,52010063002,39.694709,140.117325,0,3
3,2023-07-15 07:25:00,3,高齢者等避難,洪水,牛島西三丁目,牛島西三丁目,5,秋田県,5201,秋田市,52010063003,39.691059,140.105982,0,3
4,2023-07-15 07:25:00,3,高齢者等避難,洪水,牛島西四丁目,牛島西四丁目,5,秋田県,5201,秋田市,52010063004,39.688827,140.106109,0,3


In [125]:
# 位置参照情報が結合されていない行を確認する
evac_info_with_block_df[evac_info_with_block_df["緯度"].isnull()]

# 該当する行がなければok
# あれば、避難情報の大字町丁目名に誤字脱字や表記ゆれがないか確認する

,MAIL_TIMESTAMP,LEVEL,LEVEL_NAME,REASON,BLOCK_ORIGINAL,BLOCK,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,緯度,経度,原典資料コード,大字・字・丁目区分コード


## 避難情報のサブセット作成

マッピングする時点一覧`MAPPING_DATETIME_LIST`の値に応じて、避難情報のサブセットを作成する。


In [126]:
# MAPPING_DATETIME_LISTの値を日付型に変換
MAPPING_DATETIME_LIST_dt = pd.to_datetime(MAPPING_DATETIME_LIST)

MAPPING_DATETIME_LIST_dt

DatetimeIndex(['2023-07-15 09:00:00', '2023-07-15 11:00:00',
               '2023-07-15 13:00:00', '2023-07-15 15:00:00',
               '2023-07-15 17:00:00', '2023-07-15 19:00:00',
               '2023-07-16 05:00:00'],
              dtype='datetime64[ns]', freq=None)

In [127]:
# 与えられた日付型のlistに対して、その各日時以前の全ての避難情報をdataframeのサブセットとして返す
evac_info_with_block_df_subsets = [
    evac_info_with_block_df[evac_info_with_block_df["MAIL_TIMESTAMP"] <= dt]
    for dt in MAPPING_DATETIME_LIST_dt
]

# 各サブセットの行数を確認
for subset in evac_info_with_block_df_subsets:
    print(subset.shape[0])

49
267
304
486
547
577
611


In [128]:
# 各サブセットの`MAIL_TIMESTAMP`の最大値と最小値を確認
for subset in evac_info_with_block_df_subsets:
    print(subset["MAIL_TIMESTAMP"].min(), subset["MAIL_TIMESTAMP"].max())

2023-07-15 07:25:00 2023-07-15 07:57:00
2023-07-15 07:25:00 2023-07-15 11:00:00
2023-07-15 07:25:00 2023-07-15 12:30:00
2023-07-15 07:25:00 2023-07-15 14:57:00
2023-07-15 07:25:00 2023-07-15 16:32:00
2023-07-15 07:25:00 2023-07-15 18:14:00
2023-07-15 07:25:00 2023-07-16 04:50:00


In [129]:
def remove_duplicate(df_subset):
    """与えられたサブセットに対して、`BLOCK`列と`REASON`列の両方が重複する行のうち、
    `MAIL_TIMESTAMP`列の時刻が最も遅い（大きい）行を残し、他の行を削除する。
    """
    df_subset = df_subset.sort_values(
        "MAIL_TIMESTAMP", ascending=False
    ).drop_duplicates(["BLOCK", "REASON"])
    return df_subset


# 重複行を削除したサブセットを作成
evac_info_with_block_df_subsets = [
    remove_duplicate(subset) for subset in evac_info_with_block_df_subsets
]

# 改めて、各サブセットの行数を確認
for subset in evac_info_with_block_df_subsets:
    print(subset.shape[0])

39
137
138
138
138
165
182


## 可視化


In [130]:
import folium

### 避難情報の地理的分布

単純に、避難情報が発表された地域のマッピングを行う場合は、以下の一連のセルを実行する。

- folium を使って、各サブセットの避難情報を地図上に表示する
- その際、`LEVEL`列の値によってマーカーの色を変える。
  - `LEVEL`==3 の場合、マーカーの色は赤
  - `LEVEL`==4 の場合、マーカーの色は紫
  - `LEVEL`==5 の場合、マーカーの色は黒
  - それ以外の場合、マーカーはデフォルトの色
- また、`REASON`列の値によってマーカーの icon を変える。
  - `REASON`=="洪水"の場合、マーカーの icon は `alert`
  - `REASON`=="土砂災害"の場合、マーカーの icon は `arrow-down`

以降、各サブセットの避難情報を地図上に表示して、マップをフォルダ`maps`内に保存する。保存された地図（HTML ファイル）は、ブラウザで開くことで拡大、縮小などインタラクティブに操作可能。


In [131]:
# evac_info_with_block_df_subsets の各サブセットについて、地図描画を行う

# 保存された地図を初期化
maps_evac = []

for i, subset in enumerate(evac_info_with_block_df_subsets):
    # 地図の初期化
    m = folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM_START, tiles=MAP_TILES)

    # 会社の所在地などのマーカーを地図に追加
    for marker in MAP_MARKERS:
        folium.Marker(
            location=marker["location"],
            popup=marker["popup"],
            icon=folium.Icon(
                color=marker["color"],
                icon=marker["icon"],
                icon_color="white",
                prefix="fa",
            ),
        ).add_to(m)

    # 避難情報のマーカーを地図に追加
    for _, row in subset.iterrows():
        folium.Marker(
            location=[row["緯度"], row["経度"]],
            popup=row["BLOCK"],
            icon=folium.Icon(
                color="red"
                if row["LEVEL"] == 3
                else "purple"
                if row["LEVEL"] == 4
                else "black"
                if row["LEVEL"] == 5
                else "blue",
                icon="warning-sign"
                if row["REASON"] == "洪水"
                else "arrow-down"
                if row["REASON"] == "土砂災害"
                else "info-sign",
                icon_color="white",
            ),
        ).add_to(m)

    # 凡例を地図に追加
    m.get_root().html.add_child(
        folium.Element(
            MAP_LEGEND.replace("[[datetime_string]]", MAPPING_DATETIME_LIST[i])
        )
    )

    # 地図をhtmlファイルとして保存
    m.save(
        os.path.join(
            MAP_DIR,
            f"map_evac_{MAPPING_DATETIME_LIST[i].replace(' ', '_').replace(':', '')}.html",
        )
    )

    # 地図をリストに追加
    maps_evac.append(m)

In [132]:
# 最新の地図を表示
maps_evac[-1]

### 従業員の住所と避難情報の関係

従業員の住所と避難情報の関係を可視化する場合は、以下の一連のセルを実行する。色やアイコンの設定は、上記の「避難情報の地理的分布」と同様。


In [133]:
# evac_info_with_block_df_subsets の各サブセットについて、
# BLOCKごとのdictionaryに集約する関数


def aggregate_by_block(df_subset):
    """与えられたサブセットに対して、BLOCKごとのdictionaryに集約する関数"""

    # 出力するdictionaryの初期化
    df_subset_dict = {}

    # df_subsetの行ごとに内容を確認してdictionaryに追加していく
    for _, row in df_subset.iterrows():
        if row["BLOCK"] not in df_subset_dict:
            df_subset_dict[row["BLOCK"]] = {"避難情報": {}, "最大警戒レベル": row["LEVEL"]}
            df_subset_dict[row["BLOCK"]]["避難情報"][row["REASON"]] = {
                "MAIL_TIMESTAMP": row["MAIL_TIMESTAMP"],
                "LEVEL": row["LEVEL"],
                "LEVEL_NAME": row["LEVEL_NAME"],
            }
        else:
            # 同一のBLOCK内で異なるREASONでの避難情報が複数ある場合は、
            # 両方をdf_subset_dict[row["BLOCK"]]["避難情報"]内に格納する
            if row["REASON"] not in df_subset_dict[row["BLOCK"]]["避難情報"]:
                df_subset_dict[row["BLOCK"]]["避難情報"][row["REASON"]] = {
                    "MAIL_TIMESTAMP": row["MAIL_TIMESTAMP"],
                    "LEVEL": row["LEVEL"],
                    "LEVEL_NAME": row["LEVEL_NAME"],
                }
                # その上で、必要に応じてそのBLOCKにおける最大警戒レベルを更新する
                if row["LEVEL"] > df_subset_dict[row["BLOCK"]]["最大警戒レベル"]:
                    df_subset_dict[row["BLOCK"]]["最大警戒レベル"] = row["LEVEL"]
            # すでに`remove_duplicate()`で、同一のBLOCK内で同じREASONの避難情報が複数ある場合は、
            # MAIL_TIMESTAMPが最も遅い（大きい）ものを残しているので、ここではこれ以上の処理をしない

    return df_subset_dict


evac_info_with_block_df_subsets_dicts = [
    aggregate_by_block(subset) for subset in evac_info_with_block_df_subsets
]

# evac_info_with_block_df_subsets_dictの内容を確認
evac_info_with_block_df_subsets_dicts[0]

{'上新城五十丁': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:57:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '下新城岩城': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:57:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '下新城笠岡': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:57:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '下新城長岡': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:57:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '上新城中': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:57:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '御野場新町五丁目': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:25:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},
 '仁井田本町三丁目': {'避難情報': {'洪水': {'MAIL_TIMESTAMP': Timestamp('2023-07-15 07:25:00'),
    'LEVEL': 3,
    'LEVEL_NAME': '高齢者等避難'}},
  '最大警戒レベル': 3},


In [134]:
# evac_info_with_block_df_subsets_dicts の各要素について、
# 従業員の住所一覧に基づいた地図描画を行う

# 保存された地図を初期化
maps_employee = []

for i, evac_subset_dict in enumerate(evac_info_with_block_df_subsets_dicts):
    # 地図の初期化
    m = folium.Map(location=MAP_CENTER, zoom_start=MAP_ZOOM_START, tiles=MAP_TILES)

    # 会社の所在地などのマーカーを地図に追加
    for marker in MAP_MARKERS:
        folium.Marker(
            location=marker["location"],
            popup=marker["popup"],
            icon=folium.Icon(
                color=marker["color"],
                icon=marker["icon"],
                icon_color="white",
                prefix="fa",
            ),
        ).add_to(m)

    # `BLOCK` 列をキーにして、 evac_subset_dict の情報から、
    # employee_address_with_block_df に列として追加する
    employee_address_with_block_df["MAX_LEVEL"] = employee_address_with_block_df[
        "BLOCK"
    ].apply(
        lambda x: evac_subset_dict[x]["最大警戒レベル"] if x in evac_subset_dict.keys() else 0
    )

    employee_address_with_block_df["EVACS"] = employee_address_with_block_df[
        "BLOCK"
    ].apply(
        lambda x: "\n".join(
            [
                f"{reason}: レベル{evac_subset_dict[x]['避難情報'][reason]['LEVEL']} ({evac_subset_dict[x]['避難情報'][reason]['LEVEL_NAME']}) at {evac_subset_dict[x]['避難情報'][reason]['MAIL_TIMESTAMP']}"
                for reason in evac_subset_dict[x]["避難情報"].keys()
            ]
        )
        if x in evac_subset_dict.keys()
        else "（避難情報なし）"
    )

    # 従業員のマーカーを地図に追加
    for _, employee in employee_address_with_block_df.iterrows():
        if employee["MAX_LEVEL"] >= 0:
            # ここではemployee["MAX_LEVEL"] >= 0 とすることで全ての従業員を地図上に表示するが、
            # もし特定の警戒レベル以上の地域に住んでいる従業員のみを表示したい場合は
            # ここのif条件を調整する
            # 例：警戒レベル3以上の従業員のみを表示するには employee["MAX_LEVEL"] >= 3 とする
            folium.Marker(
                location=[employee["緯度"], employee["経度"]],
                popup=f"{employee['ID']} {employee['NAME']}:\n{employee['EVACS']}",
                icon=folium.Icon(
                    color="red"
                    if employee["MAX_LEVEL"] == 3
                    else "purple"
                    if employee["MAX_LEVEL"] == 4
                    else "black"
                    if employee["MAX_LEVEL"] == 5
                    else "lightgray",
                    icon="warning-sign" if employee["MAX_LEVEL"] > 0 else "user",
                    icon_color="white",
                ),
            ).add_to(m)

    # 凡例を地図に追加
    m.get_root().html.add_child(
        folium.Element(
            MAP_LEGEND.replace("[[datetime_string]]", MAPPING_DATETIME_LIST[i])
        )
    )

    # 地図をhtmlファイルとして保存
    m.save(
        os.path.join(
            MAP_DIR,
            f"map_employee_{MAPPING_DATETIME_LIST[i].replace(' ', '_').replace(':','')}.html",
        )
    )

    # 地図をリストに追加
    maps_employee.append(m)

In [135]:
# 最新の地図を表示
maps_employee[-1]